In [ ]:
# T1-clusterScore
===========================================================

In [ ]:
# 可跑版 使用angle-bert-base-uncased-nli-en-v1进行single-pass  ByTitle
# update:加了评估，可对第一天结果进行评估与记录评估结果
# updata:ByTitle
# update：使用angle加载
# updata:to .py :single-pass-ByTitle-angle-bert-AngleLOAD-Eval.py
# update:保存predict_clusters
# update:3.6 load FIX data
# 暂时聚类最佳:angle-bert ANGLE_LOAD thresold = 0.975
# update：3.9 新增Score计算（以angle-bert ANGLE_LOAD thresold = 0.975为例）
import pandas as pd
# from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoModel, AutoTokenizer
from angle_emb import AnglE


# yes! 聚类评估！！！可跑 TP, FP, TN, FN 得到RI、Precision、Recall、F1，ARI
# update:单个成簇的处理
from itertools import combinations
from math import comb


In [ ]:
# 使用angle加载
model_id = '../NewsAthmTask2/models/angle-bert-base-uncased-nli-en-v1'
angle = AnglE.from_pretrained(model_id, pooling_strategy='cls_avg').cuda()

# 加载数据
# data = pd.read_csv('Data231202-231211.csv')
# data = pd.read_csv('./Data231202-231211_FIX/Data231202_newDATA.csv')
data = pd.read_csv('./Data231202-231211/Data231202.csv')


# 将日期转换为日期时间格式
data['pub_time'] = pd.to_datetime(data['pub_time'])

# 获取唯一日期列表
dates = data['pub_time'].dt.date.unique()

# 定义聚类中心更新函数
def update_cluster_center(cluster):
    # cluster_embeddings = sbert_model.encode(cluster)
    cluster_embeddings = angle.encode(cluster, to_numpy=True) # 使用angle加载
     
    return np.mean(cluster_embeddings, axis=0)
# 设置阈值
threshold = 0.972

# 对于每个日期
cluster_results = []
cnt = 0
for date in dates:
    print(cnt)
    cnt+=1
    # 获取该日期的新闻标题
    news_data = data[data['pub_time'].dt.date == date]['title'].tolist()
    # 获取该日期的新闻正文
    # news_data = data[data['pub_time'].dt.date == date]['body'].tolist() # ByBody

    # 使用SBERT模型获取语义向量
    # embeddings = sbert_model.encode(news_data)

    embeddings = angle.encode(news_data, to_numpy=True) # 使用angle加载

    # 定义当天的簇列表
    daily_clusters = []

    # 对于每个新闻数据
    # for i, embedding in enumerate(data_vec):
    for i, embedding in enumerate(embeddings):
        # 如果簇列表为空，则新开一个簇
        if not daily_clusters:
            # daily_clusters.append({'center': embedding, 'members': [news_data[i]]})
            daily_clusters.append({'center': embedding, 'members': [i],'news':[news_data[i]]}) # 改为存index
            continue

        # 计算当前数据点与各个簇中心的相似度
        # similarities = [cosine_similarity([embedding], [cluster['center']])[0][0] for cluster in daily_clusters]
        similarities = [cosine_similarity([embedding], [cluster['center']])[0][0] for cluster in daily_clusters]

        # 找到最大相似度及其对应的簇索引
        max_similarity = max(similarities)
        max_index = similarities.index(max_similarity)

        # 如果最大相似度大于阈值，则将当前数据点加入对应簇，并更新簇中心
        if max_similarity > threshold:
            # daily_clusters[max_index]['members'].append(news_data[i])
            daily_clusters[max_index]['members'].append(i) # 改为存index
            daily_clusters[max_index]['news'].append(news_data[i]) # 改为存index
            # daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['members'])
            # daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['members'],news_data)
            daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['news'])
        # 否则新开一个簇
        else:
            # daily_clusters.append({'center': embedding, 'members': [news_data[i]]})
            daily_clusters.append({'center': embedding, 'members': [i],'news':[news_data[i]]}) # 改为存index

    # 将当天的簇信息添加到结果列表中
    cluster_results.append({'date': date, 'clusters': daily_clusters})

# 评估
#    true_clusters = [[0],[1],[2,16],[3],[4,6,22,50,73,87],[5],[7],[8,61],[9],[10,77],[11],[12],[13],
# [14,29,41,51,59,67,78,84],[15],[17],[18],[19],[20],[21,68],[23],[24],[25],[26],
# [27],[28],[30],[31],[32],[33],[34],[35,55],[36],[37],[38],[39],[40],[42],[43,64],
# [44],[45],[46],[47,53,88],[48],[49],[52],[54],[56],[57],[58],[60],[62],[63],[65],
# [66],[69],[70],[71],[72],[74],[75],[76],[79],[80],[81],[82],[83],[85],[86],
# [89],[90],[91],[92],[93],[94],[95]]

# update:3.6 新true_clusters for FIX
# true_clusters = [[0],[1,4,6,23,28,41],[2],[3],[5],[7],[8],[9,31],[10],[11],[12],[13],[14],[15],[16],[18],[19],[20],[17,21,38],[22],[24],[25],[26],[27],[29],[30],[32],[33],[34],[35],[36],[37],[39],[40],[42],[43],[44],[45],[46],[47],[48,49],[50],[51],[52],[53],[54],[55],[56],[57],[58],[59],[60],[61],[62],[63],[64],[65],[66],[67],[68,70,74],[69],[71],[72],[73],[75],[76],[77],[78],[79],[80]]


predicted_clusters = []
for cluster in cluster_results[0]['clusters']: # 2023-12-02的簇s
    clus_index = []
    for i in cluster['members']:
        clus_index.append(i)
    predicted_clusters.append(clus_index)
print(predicted_clusters)


In [ ]:
# 现在我有一个列表如下所示[[0], [1], [2], [3], [4, 87], [5], [6, 23], [7], [8], [9], [10, 77], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21, 68], [22, 50], [24], [25], [26], [27], [28], [29, 78], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 43], [41, 59], [42], [44], [45], [46], [47, 88], [48], [49], [51, 67], [52], [53], [54], [55], [56], [57], [58], [60], [61], [62], [63], [64], [65], [66], [69], [70], [71], [72], [73], [74], [75], [76], [79], [80], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [94], [95]]
# ，列表中每个子列表表示一个聚类的簇，子列表的元素个数即为簇中元素个数，然后我有一个语料文件'../Data231202-231211/Data231202.csv'，现在这个列表中的每个子列表的元素为该语料文件中的语料索引，现在我需要新增一列clus_news_num用来记录每个语料对应的簇中的元素个数，并将整个语料问价按语料对应的簇中的元素个数进行从大到小排序，要求同样大小的排名相同，然后在新增一列S_scale，该列内容为根据语料对应的簇中的元素个数进行归一化后的数，还有一列S_score为20*S_scale,还有一列index表示语料原始的坐标，最后将dataframe中的index，title,body,clus_news_num,S_scale，S_score这几列存入到新的csv文件中，请给出完整详细的代码
# update:3.9 可跑！！
from sklearn.preprocessing import MinMaxScaler

# 已有的聚类结果
# clusters = [[0], [1], [2], [3], [4, 87], [5], [6, 23], [7], [8], [9], [10, 77], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21, 68], [22, 50], [24], [25], [26], [27], [28], [29, 78], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 43], [41, 59], [42], [44], [45], [46], [47, 88], [48], [49], [51, 67], [52], [53], [54], [55], [56], [57], [58], [60], [61], [62], [63], [64], [65], [66], [69], [70], [71], [72], [73], [74], [75], [76], [79], [80], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [94], [95]]
clusters = predicted_clusters

# 创建一个字典，键是语料索引，值是对应的簇大小
index_to_cluster_size = {index: len(cluster) for cluster in clusters for index in cluster}

# 读取语料文件
df = pd.read_csv('./Data231202-231211/Data231202.csv')

# 新增列clus_news_num，记录每个语料对应的簇的大小
df['clus_news_num'] = df.index.map(index_to_cluster_size)

# 根据簇大小进行排序，并添加排名，相同大小的排名相同
df = df.sort_values(by='clus_news_num', ascending=False)
df['rank'] = df['clus_news_num'].rank(method='min', ascending=False)

# 新增列S_scale，为簇大小的归一化结果
scaler = MinMaxScaler()
df['S_scale'] = scaler.fit_transform(df[['clus_news_num']])

# 新增列S_score，为S_scale的值乘以20
df['S_score'] = df['S_scale'] * 20

# 新增列index，表示语料原始的坐标
df['ori_indexFrom0'] = df.index

# 只保留需要的列，并保存到新的CSV文件
final_df = df[['ori_indexFrom0', 'title', 'body', 'clus_news_num', 'rank','S_scale', 'S_score']]
final_df.to_csv('./T1ClusterScore/final_result.csv', index=False)
print("FINISH!")

In [ ]:
# T1-clusterScore
# df['S_score']
===========================================================

In [ ]:
# T2-WebsiteScore
===========================================================

In [ ]:
# 40个网站的排名以及赋分结果在./T2WebsiteRank/website_Rank_new.csv
# Data231202-231211/Data231202.csv
# 读取Data231202-231211/Data231202.csv，其中的website_id为网站id，现在读取./T2WebsiteRank/website_Rank_new.csv，该文件存有website_id对应的S_task_web，现在需要将Data231202.csv中的每个语料对应的website_id对应的S_task_web新增一列进行存储，然后根据S_task_web进行排序，允许并列，新增rank列，将结果中website_id,title,S_task_web,rank存到新的csv文件
import pandas as pd

# 读取两个csv文件
data_df = pd.read_csv('./Data231202-231211/Data231202.csv')
rank_df = pd.read_csv('./T2WebsiteRank/website_Rank_new.csv')

# 将两个DataFrame合并
merged_df = pd.merge(data_df, rank_df, on='website_id')

# 根据S_task_web列进行排序，并添加排名，相同权重的排名相同
merged_df = merged_df.sort_values(by='S_task_web', ascending=False)
merged_df['rank'] = merged_df['S_task_web'].rank(method='min', ascending=False)

# 只保留需要的列，并保存到新的CSV文件
final_df = merged_df[['website_id', 'title', 'S_task_web', 'rank']]
final_df.to_csv('./T2WebsiteRank/Data231202_scoreResult.csv', index=False)

In [ ]:
# T2-WebsiteScore
# merged_df['S_task_web']
===========================================================

In [ ]:
# T3-BodyLengthScore
===========================================================

In [ ]:
import pandas as pd

# 读取CSV文件
input_file = './Data231202-231211_FIX/Data231202_newDATA.csv'  # 替换为你的输入文件路径
output_file = './T3BodyLenRank/Data231202_newDATA_rank_Score.csv'  # 替换为你的输出文件路径

# 读取CSV文件并计算正文长度
df = pd.read_csv(input_file)
df['body_len'] = df['body'].apply(lambda x: len(str(x).split()))  # 假设每个单词之间用空格分隔

# 按正文长度进行排序
df = df.sort_values(by='body_len', ascending=False)

# 添加排名列
df['rank_len'] = df['body_len'].rank(method='min', ascending=False)

# 计算S_scale并添加列
max_len = df['body_len'].max()
min_len = df['body_len'].min()
df['S_scale'] = (df['body_len'] - min_len) / (max_len - min_len)

# 计算body_len_score并添加列
df['body_len_score'] = 20 * df['S_scale']

# 保存结果到新的CSV文件
df.to_csv(output_file, index=False)

print("处理完成，并将结果保存到新的CSV文件中。")


In [ ]:
# T3-BodyLengthScore
# df['body_len_score']
===========================================================

In [ ]:
# T4-TitleTextRankfScore
===========================================================

In [ ]:
# 能跑！
# 用这个！ 
# 3.9：还需检查textrank算法合理性，根据结果调整窗口大小，加上标题权重处理
# update：加标题权重处理 标题权重为标题中每个词对应的词权重之和
# 我现在有一个word_weight.csv文件保存的是词的权重，里面有word，weight两列分别对应词和权重，现在我需要读取‘./Data231202-231211/Data231202.csv’文件，文件中title列为新闻标题，我需要对每个title列元素进行处理，首先对其进行分词，然后找到每个词在word_weight.csv中对应的权重，将标题中所有词的权重相加得到该标题的权重，对title列的每个标题都进行处理得到每个标题的权重，然后根据标题权重进行从大到小排序，为其添加一个排名（从1开始），要求相同权重的排名相同，然后将结果存入到新的csv文件'titles_weight.csv'中，请给出详细完整的代码
# 将标题权重归一化映射到分数！
import pandas as pd
import networkx as nx
from collections import defaultdict
from nltk.tokenize import word_tokenize # 使用NLTK进行分词，根据需要替换为适合孟加拉语的分词方法

import spacy
# from gensim.summarization import keywords
from collections import defaultdict
# import spacy
import bn_core_news_sm
from sklearn.preprocessing import MinMaxScaler # 归一化
import matplotlib.pyplot as plt


# import pytextrank
# =======
# 去除停用词
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import string
# ====================

In [ ]:
# 加载孟加拉语模型
nlp = bn_core_news_sm.load()
# # textrank算法计算权重
# def textrank_weighted_word_graph(merged_titles):
#     # tokens = word_tokenize(merged_titles)  # 根据实际情况替换分词方法
#     tokens = nlp(merged_titles) # 分词
#     print(len(tokens))
#     print(tokens)
    
#     graph = nx.Graph()
#     window_size = 80  # 根据需要调整窗口大小  # todo:调整窗口

#     for i, token in enumerate(tokens):
#         for j in range(i+1, min(i+window_size, len(tokens))):
#             if token != tokens[j]:  # 添加边，避免自环
#                 graph.add_edge(token, tokens[j], weight=1)

#     # 使用NetworkX的PageRank算法计算每个节点（词）的权重
#     pagerank_scores = nx.pagerank(graph, weight='weight')
#     return pagerank_scores

# update 3.9：改进版！！
def textrank_weighted_word_graph(merged_titles):
    tokens = nlp(merged_titles) # 分词
    print(len(tokens))
    print(tokens)
    
    graph = nx.Graph()
    window_size = 80  # 根据需要调整窗口大小
    
    for i, token in enumerate(tokens):
        for j in range(i+1, min(i+window_size+1, len(tokens))):
            if token != tokens[j]:  # 添加边,避免自环
                if graph.has_edge(token, tokens[j]):
                    graph[token][tokens[j]]['weight'] += 1 #在添加边时,先检查边是否已经存在。如果边已经存在,则将权重加1;否则,添加一个新边,权重为1。这样可以避免重复添加边。
                else:
                    graph.add_edge(token, tokens[j], weight=1)
    
    # 使用NetworkX的PageRank算法计算每个节点（词）的权重
    pagerank_scores = nx.pagerank(graph, weight='weight')

    return pagerank_scores,graph

# 读取CSV文件并合并所有标题
# df = pd.read_csv('your_file.csv')
# df = pd.read_csv('./Data231202-231211_FIX/Data231202_newDATA.csv')
df = pd.read_csv('./Data231202-231211/Data231202.csv')

# merged_titles = ' '.join(df['title'].replace("\n", ""))
merged_titles = ' '.join(title.strip() for title in df['title'])

# ====================================
# 获取孟加拉语的停用词列表
stop_words = set(stopwords.words('bengali'))
# print(stop_words)

# 自定义标点符号列表
custom_punctuation = ['‘', '’']

# 合并 NLTK 提供的标点符号列表和自定义标点符号列表
all_punctuation = string.punctuation + ''.join(custom_punctuation)

print(all_punctuation)
# 分词# word_tokens = word_tokenize(merged_titles)

word_tokens = nlp(merged_titles) # 分词
# word_tokens = merged_titles.split() # 根据空格分词
token_texts = [token.text.strip() for token in word_tokens] # 去除多余空格

# print(token_texts)
print(type(token_texts))




# 去除停用词
# filtered_titles = [w for w in word_tokens if not w in stop_words]
filtered_titles = [w for w in token_texts if not w in stop_words] # 去除停用词
filtered_titles = [word for word in filtered_titles if word not in all_punctuation] # 去除标点符号

print("filtered_titles len\n",len(filtered_titles)) # 字符串数量！


# 将去除停用词后的词重新组合成字符串
# filtered_titles_text = ' '.join(filtered_titles)
filtered_titles_text = ' '.join(filtered_titles)

# print(filtered_titles_text)
print(len(filtered_titles_text)) # 字符串长度！别被误导（所少个字符）
# ====================================

# 计算词权重
# word_weights = textrank_weighted_word_graph(merged_titles)
word_weights,graph = textrank_weighted_word_graph(filtered_titles_text)

# 保存pagerank算法后的词关系权重 可视化
# 根据PageRank值更新边的权重
pagerank_weighted_graph = nx.Graph()
for node, score in word_weights.items():
    pagerank_weighted_graph.add_node(node)

for u, v, data in graph.edges(data=True):
    weight = data['weight'] * word_weights[u] * word_weights[v]
    pagerank_weighted_graph.add_edge(u, v, weight=weight)

with open('./T4TitleTextRank/graph_content.txt', 'w') as file:
    file.write(str(nx.to_dict_of_dicts(pagerank_weighted_graph)))

sorted_words = sorted(word_weights.items(), key=lambda x: x[1], reverse=True)

# 保存到新的CSV文件
# word_weights_df = pd.DataFrame(word_weights.items(), columns=['word', 'weight'])
word_weights_df = pd.DataFrame(sorted_words, columns=['word', 'weight'])


# word_weights_df.to_csv('./T4TitleTextRank/word_weight.csv', index=False)
word_weights_df.to_csv('./T4TitleTextRank/word_weight_new.csv', index=False)

# 接下来，计算每个标题的权重
# 读取词权重文件
# word_weights_df = pd.read_csv('./T4TitleTextRank/word_weight.csv')
word_weights_df = pd.read_csv('./T4TitleTextRank/word_weight_new.csv')

# 将词权重转换为字典，方便查找
word_weights = pd.Series(word_weights_df.weight.values, index=word_weights_df.word).to_dict()

# 读取新闻标题文件
titles_df = pd.read_csv('./Data231202-231211/Data231202.csv')
# titles_df = titles_df['title']


# 定义一个函数，用于计算标题的权重
def calculate_title_weight(title):
    doc = nlp(title)
    # 对标题进行分词并计算总权重
    # return sum(word_weights.get(token.text, 0) for token in doc)  # 如果词不在word_weights中，则默认权重为0
    return sum(word_weights.get(token.text, 0) for token in doc if token.text not in stop_words and token.text not in string.punctuation)  # 如果词不在word_weights中，则默认权重为0


# 计算每个标题的权重
titles_df['weight'] = titles_df['title'].apply(calculate_title_weight)

# 根据权重排序并添加排名，相同权重的排名相同
titles_df = titles_df.sort_values(by='weight', ascending=False)
titles_df['rank'] = titles_df['weight'].rank(method='min', ascending=False)

# 对权重进行归一化处理，并存储结果到"S_scale"列
scaler = MinMaxScaler()
titles_df['S_scale'] = scaler.fit_transform(titles_df[['weight']])  # 归一化映射到分数！

# 创建"S_score"列
titles_df['S_score'] = titles_df['S_scale'] * 20

# 只保留需要的列
final_df = titles_df[['title', 'weight', 'rank', 'S_scale', 'S_score']]


# 保存到新的csv文件
# final_df.to_csv('./T4TitleTextRank/titles_weight.csv', index=False)
final_df.to_csv('./T4TitleTextRank/titles_weight_new.csv', index=False)

In [ ]:
# T4-TitleTextRankfScore
# titles_df['S_score']
===========================================================

In [ ]:
# T5-CategoryScore
===========================================================

In [ ]:
# 提取新闻的category1进行类别评分
import pandas as pd

# Load the original CSV file with category data
category_df = pd.read_csv('./T5CateforyScore/category_score.csv')

# Load the CSV file with news data
news_df = pd.read_csv('./Data231202-231211_FIX/Data231202_newDATA.csv')

# Merge the two DataFrames based on the "category1" column
merged_df = pd.merge(news_df, category_df, how='left', left_on='category1', right_on='category')

# Sort the merged DataFrame based on the "rank" column
sorted_df = merged_df.sort_values(by='rank')

# Select the desired columns
selected_columns = ['title', 'category1', 'rank', 'S_scale', 'S_score']
result_df = sorted_df[selected_columns]

# Save the result to a new CSV file
result_df.to_csv('./T5CateforyScore/Data231202_categoryScore.csv', index=False)


In [ ]:
# T5-CategoryScore
# result_df['S_score']
===========================================================

In [ ]:
# MergeFiveDScores
===========================================================